In [1]:
import tensorflow as tf
import os, shutil

print(tf.__version__)

2.6.1


In [2]:
MODEL_LOCATION = 'export/fire_model'  # will be created

IMG_HEIGHT = 448  # same value as used while model training
IMG_WIDTH = IMG_HEIGHT
IMG_CHANNELS = 3
CLASS_NAMES = 'Fire No-Fire'.split()

CHECK_POINT_DIR='gs://fire_detection_anurag/models/accuracy/chkpts'

In [3]:
model = tf.keras.models.load_model(CHECK_POINT_DIR)

Set signature of serving function

In [4]:
def preprocess_step1(filename):
    img_bytes = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img_bytes,
                               channels=IMG_CHANNELS)
    img = tf.image.convert_image_dtype(img, tf.float32)
    
    return tf.image.resize_with_pad(img, IMG_HEIGHT, IMG_WIDTH)


@tf.function(input_signature=[tf.TensorSpec([None,],
                                            dtype=tf.string)])
def preprocess_step2(filenames):
    input_images = tf.map_fn(
                             preprocess_step1,
                             filenames,
                             fn_output_signature=tf.float32
                            )
    
    return {
            'input_images': input_images
           }


@tf.function(input_signature=[tf.TensorSpec([None,
                                             IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS],
                                            dtype=tf.float32)])
def get_model_pred(input_images):
    batch_pred = model(input_images) # same as model.predict()
    top_prob = tf.math.reduce_max(batch_pred,
                                  axis=[1])
    pred_label_index = tf.math.argmax(batch_pred,
                                      axis=1)
    pred_label = tf.gather(tf.convert_to_tensor(CLASS_NAMES),
                           pred_label_index)
    
    return {
            'probability': top_prob,
            'image_type_int': pred_label_index,
            'image_type_str': pred_label
           }


@tf.function(input_signature=[tf.TensorSpec([None,],
                                            dtype=tf.string)])
def predict_filename(filenames):
    preproc_output = preprocess_step2(filenames)
    
    return get_model_pred(**preproc_output)

In [5]:
shutil.rmtree('export',
              ignore_errors=True)

os.mkdir('export')

model.save(MODEL_LOCATION,
           signatures={
                       'serving_default': predict_filename,
                      }
          )

INFO:tensorflow:Assets written to: export/fire_model/assets


Check model signature

In [6]:
!saved_model_cli show --dir {MODEL_LOCATION} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['filenames'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: serving_default_filenames:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['image_type_int'] tensor_info:
        dtype: DT_INT64
        shape: (-1)
        name: StatefulPartitionedCall:0
    outputs['image_type_str'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: StatefulPartitionedCall:1
    outputs['probability'] tensor_info:
        dty

In [7]:
!pwd

/home/jupyter/Computer_Vision/fire_detection/Inferencing/pub_sub


## Deploy model to Vertex-AI endpoint

In [8]:
!gsutil -m cp -r export/* gs://fire_detection_anurag/models/pub_sub_beam/export/

Copying file://export/fire_model/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://export/fire_model/keras_metadata.pb [Content-Type=application/octet-stream]...
Copying file://export/fire_model/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://export/fire_model/variables/variables.index [Content-Type=application/octet-stream]...
/ [4/4 files][ 10.9 MiB/ 10.9 MiB] 100% Done                                    
Operation completed over 4 objects/10.9 MiB.                                     


**Note**: While choosing appropriate 'MODEL_NAME', make sure there is no already deployed model (on Vertex-AI) having the exact same name.

In [13]:
%%writefile vertex_deploy.sh
REGION="us-central1"
ENDPOINT_NAME="fire_endpoint"
MODEL_NAME="fire_detector_pubsub"
MODEL_LOCATION="gs://fire_detection_anurag/models/pub_sub_beam/export/fire_model"
IMAGE_URI="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-5:latest"

for i in "$@"
do
case $i in
        -r=*|--region=*) REGION="${i#*=}"; shift ;;
        -e=*|--endpoint_name=*) ENDPOINT_NAME="${i#*=}"; shift ;;
        -m=*|--model_name=*) MODEL_NAME="${i#*=}"; shift ;;
        -l=*|--model_location=*) MODEL_LOCATION="${i#*=}"; shift ;;
        -i=*|--image_uri=*) IMAGE_URI="${i#*=}"; shift ;;
        *) echo "Unknown parameter passed: $1"; exit 1 ;;
esac
done

echo "Deploying model $MODEL_NAME"

if [[ $(gcloud ai endpoints list --region=$REGION --format="value(display_name)" | grep $ENDPOINT_NAME) ]]; then
    echo "The endpoint named $ENDPOINT_NAME already exists."
else
    # Create endpoint.
    echo "Creating $ENDPOINT_NAME endpoint now."
    gcloud ai endpoints create \
      --region=$REGION \
      --display-name=$ENDPOINT_NAME
fi

ENDPOINT_ID=$(gcloud ai endpoints list --region=$REGION --format="value(name)" --filter="displayName=$ENDPOINT_NAME")
echo "The endpoint_id is $ENDPOINT_ID"

if [[ $(gcloud ai models list --region=$REGION --format="value(display_name)" | grep $MODEL_NAME) ]]; then
    echo "The model named $MODEL_NAME already exists."
else
    # Upload model.
    echo "Uploading $MODEL_NAME model now."
    gcloud ai models upload \
      --region=$REGION \
      --display-name=$MODEL_NAME \
      --container-image-uri=$IMAGE_URI \
      --artifact-uri=$MODEL_LOCATION
fi

MODEL_ID=$(gcloud ai models list --region=$REGION --format="value(name)" --filter="displayName=$MODEL_NAME")
echo "The model_id is $MODEL_ID"

echo "Deploying model now"
gcloud ai endpoints deploy-model $ENDPOINT_ID\
  --region=$REGION \
  --model=$MODEL_ID \
  --display-name=$MODEL_NAME \
  --traffic-split=0=100

Overwriting vertex_deploy.sh


In [14]:
!bash vertex_deploy.sh

Deploying model fire_detector_pubsub
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
The endpoint named fire_endpoint already exists.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
The endpoint_id is 6569494015031377920
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Uploading fire_detector_pubsub model now.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [7204935938303066112]...done.                            
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
The model_id is projects/9118975290/locations/us-central1/models/13304090696089600
Deploying model now
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [7398590722279997440]...done.                            
Deployed a model to the endpoint 6569494015031377920. Id of the deployed model: 668687787641274368.
